<a href="https://colab.research.google.com/github/Mhammad2022Ibrahim/A-project-API-for-Facebook/blob/master/Medical_chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [65]:
df = pd.read_csv('Medical_knowledge_base.csv', encoding='ISO-8859-1')
df.head()


,name,desc,prevent,cause,symptom,acompany,cure_department,cure_way
0,alveolar proteinosis,"alveolar proteinosis (pap), also known as rose...","1, to avoid infection with mycobacterial disea...","the cause is unknown, and it is speculated tha...","['cyanosis', 'chest pain', 'breathing difficul...",['multiple lung infections'],"['internal medicine', 'respiratory medicine']",['bronchoalveolar lavage']
1,pertussis,pertussis (whoopingcough) is an acute respirat...,"1, control of the source of infection: in the ...",(1) causes of the disease the pathogen is b. p...,"['hanging sound when inhaling', 'sexual cough'...",['lung atelect'],"['pediatrics', 'pediatric medicine']","['medical treatment', 'supportive treatment']"
2,benzene poisoning,benzene is an aromatic hydrocarbon compound ob...,"for patients with acute poisoning, you can imm...",inhalation of benzene vapor or skin contact wi...,"['disgusting', 'twitching', 'feeling obstacle']",['anemia'],['emergency department'],"['medical treatment', 'supportive treatment']"
3,asthmatic bronchitis,"asthmatoid bronchitis, also known as asthmatic...","according to the above section, for children w...",because a variety of viral and bacterial infec...,"['shrubbing's wheezing', 'wheeling sound', 'ci...",['bronchial asthma'],"['internal medicine', 'respiratory medicine']","['medical treatment', 'supportive treatment']"
4,adult respiratory distress syndrome,"adult respiratory distress syndrome, abbreviat...",patients with high-risk should be closely obse...,suppurative infections can cause bacterial tox...,"['difficulty breathing', 'purple', 'cardiac re...",['bacterial pneumonia'],"['internal medicine', 'respiratory medicine']","['medical treatment', 'supportive treatment']"


In [66]:
desc = df['desc']
symptom = df['symptom']
input_text = desc + ' .And I have this symptoms: ' + symptom
with open('input_text.txt', 'w') as f:
    for text in input_text:
        f.write(text + '\n')


In [67]:
import tensorflow_hub as hub
model = hub.load("https://tfhub.dev/prvi/tf2nq/1")

In [68]:
def post_process(model_output,n_keep=100):
    pos_logits,ans_logits,long_mask,short_mask,cross = model_output
    long_span_logits =  pos_logits
    mask = tf.cast(tf.expand_dims(long_mask,-1),long_span_logits.dtype)
    long_span_logits = (1-mask)*tf.float32.min + mask*long_span_logits
    # Get top span positions
    top_long_indices = tf.argsort(long_span_logits, direction="DESCENDING")
    top_long_indices = tf.unravel_index(top_long_indices[:n_keep], tf.shape(long_span_logits))
    top_long_indices = tf.sort(tf.transpose(top_long_indices)).numpy()
    answers = []
    for start, end in top_long_indices:
        long_span = {
            "start_token": start.numpy(),
            "end_token": end.numpy(),
            "score": long_span_logits[start, end].numpy(),
            "cure_way": df['cure_way'][start.numpy()]
        }
        answers.append(long_span)
    return answers


In [69]:
print(model.signatures.keys())


KeysView(_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(data_len, input_word_ids, question_len) at 0x7FB27D1CB340>}))


In [70]:
from collections import Counter
import nltk
nltk.download('punkt')

# Tokenize the data
tokens = []
for text in input_text:
    tokens += nltk.word_tokenize(text)

# Create the vocabulary
vocab_count = Counter(tokens)
vocab = {}
for i, (word, count) in enumerate(vocab_count.most_common()):
    vocab[word] = i + 1  # Reserve 0 for padding

UNK_TOKEN = '[UNK]'
vocab = {'word1': 1, 'word2': 2, UNK_TOKEN: len(vocab)+1}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [76]:
# Load the input text from file
with open('input_text.txt', 'r') as f:
    input_text = f.read().splitlines()

# Process each input separately
for text in input_text:
    # Run the chatbot model
    data_len = tf.constant([len(text.split())])
    input_words = text.split()
    input_word_ids = []
    for word in input_words:
      if word in vocab:
        input_word_ids.append(vocab[word])
      else:
        input_word_ids.append(vocab[UNK_TOKEN])

    question_len = tf.constant([len(str(symptom).split())])

    model_output = model.signatures['serving_default'](
    data_len=tf.constant(data_len),
    input_word_ids=tf.constant(input_word_ids),
    question_len=tf.constant(question_len))

    # Post-process the output to get the answers
    answers = post_process(model_output)


InvalidArgumentError: ignored

In [ ]:
# Print the answer with the highest score
if len(answers) > 0:
  print(f"Question: {text}")
  print(f"Answer: {answers[0]['cure_way']}")
else:
  print(f"No answer found for question: {text}")